In [1]:
#Install dependencies if not already installed
!pip install fastestimator ~/repos/fledge

Processing /home/itrushkin/repos/fledge


  Created wheel for fledge: filename=fledge-0.20-py3-none-any.whl size=168551 sha256=e2fe6d7bd2feb696d60982f43c1d73577144a9bf024e479be547056c7f383723
  Stored in directory: /tmp/pip-ephem-wheel-cache-kdt2lqb2/wheels/3a/30/9d/a88101effd240e4a8721301bebf95a245eaf4904b55b7ae530
Successfully built fledge
  Attempting uninstall: fledge
    Found existing installation: fledge 0.20
    Uninstalling fledge-0.20:
      Successfully uninstalled fledge-0.20
You should consider upgrading via the '/home/itrushkin/.virtualenvs/fledge/bin/python -m pip install --upgrade pip' command.


In [2]:
import fastestimator as fe
from fastestimator.architecture.pytorch import LeNet
from fastestimator.backend import to_tensor, argmax
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop import Average
from fastestimator.op.tensorop.gradient import Watch, FGSM
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util import ImgData, to_number
from fledge.native.fastestimator import FederatedFastEstimator
from fastestimator.dataset.data import cifar10
from fastestimator.trace.adapt import LRScheduler
from fastestimator.op.numpyop.univariate import Normalize, ChannelTranspose

batch_size=128

train_data, eval_data = cifar10.load_data()
test_data = eval_data.split(0.5)

pipeline = fe.Pipeline(train_data=train_data,
                     eval_data=eval_data,
                     test_data=test_data,
                     batch_size=batch_size,
                     ops=[Normalize(inputs="x", outputs="x",
                             mean=(0.4914, 0.4822, 0.4465), 
                             std=(0.2471, 0.2435, 0.2616)),
                          ChannelTranspose(inputs="x", outputs="x")])
model = fe.build(model_fn=lambda: LeNet(input_shape=(3, 32, 32)), \
                         optimizer_fn="adam", model_name="adv_model")

network = fe.Network(ops=[
                Watch(inputs="x"),
                ModelOp(model=model, inputs="x", outputs="y_pred"),
                CrossEntropy(inputs=("y_pred", "y"), outputs="base_ce"),
                FGSM(data="x", loss="base_ce", outputs="x_adverse", epsilon=0.04),
                ModelOp(model=model, inputs="x_adverse", outputs="y_pred_adv"),
                CrossEntropy(inputs=("y_pred_adv", "y"), outputs="adv_ce"),
                Average(inputs=("base_ce", "adv_ce"), outputs="avg_ce"),
                UpdateOp(model=model, loss_name="avg_ce")
            ])
def lr_schedule(step):
    lr = 1 / (100 * step)
    return lr

estimator = fe.Estimator(pipeline=pipeline,
                         network=network,
                         epochs=1,
                         traces=[Accuracy(true_key="y", pred_key="y_pred", output_name="clean_accuracy"),
                                Accuracy(true_key="y", pred_key="y_pred_adv", output_name="adversarial_accuracy"),
                                LRScheduler(model=model, lr_fn=lr_schedule)],
                         max_train_steps_per_epoch=None,
                         max_eval_steps_per_epoch=None,
                         monitor_names=["base_ce", "adv_ce"],
                         log_steps=50)
fledge_estimator = FederatedFastEstimator(estimator, rounds=10)
model=fledge_estimator.fit()

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── tfmnist_inmemory.py
├── agg_to_col_two_signed_cert.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── network.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── aggregator.yaml
│   │   ├── task_runner.yaml
│   │   └── data_loader.yaml
│   └── plan.yaml
├── requirements.txt
├── logs
├── agg_to_col_one_signed_cert.zip
├── cert
│   ├── config
│   │   ├── client.conf
│   │   ├── signing-ca.conf
│   │   ├── server.conf
│   │   └── root-ca.conf
│   ├── client
│   │   ├── col_one.crt
│   │   ├── col_two.csr
│   │   ├── col_two.crt
│   │   ├── col_two.key
│   │   ├── col_one.csr
│   │   └── col_one.ke

traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc809223c8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc80933320>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fbc809337b8>]


traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc809223c8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc80933320>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fbc809337b8>]


traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc809223c8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fbc80933320>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fbc809337b8>]


/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:172: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


...

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 882; num_device: 3; logging_interval: 50; 
FastEstimator-Train: step: 900; base_ce: 1.9372863; adv_ce: 2.058701; avg_ce: 1.9979937; steps/sec: 107.57; adv_model_lr: 1.1111111e-05; 
FastEstimator-Train: step: 950; base_ce: 1.8741379; adv_ce: 1.9899442; avg_ce: 1.932041; steps/sec: 35.01; adv_model_lr: 1.0526316e-05; 
FastEstimator-Train: step: 980; epoch: 10; epoch_time: 2.93 sec; 
FastEstimator-Eval: step: 980; epoch: 10; base_ce: 1.929286; adv_ce: 2.055

FastEstimator-Test: step: 980; epoch: 11; clean_accuracy: 0.336; adversarial_accuracy: 0.2616; 


FastEstimator-Test: step: 882; epoch: 10; clean_accuracy: 0.3344; adversarial_accuracy: 0.2696; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 882; num_device: 3; logging_interval: 50; 
FastEstimator-Train: step: 900; base_ce: 2.011351; adv_ce: 2.1371007; avg_ce: 2.074226; steps/sec: 100.85; adv_model_lr: 1.1111111e-05; 
FastEstimator-Train: step: 950; base_ce: 1.8855343; adv_ce: 2.0135598; avg_ce: 1.949547; steps/sec: 35.41; adv_model_lr: 1.0526316e-05; 
FastEstimator-Train: step: 980; epoch: 10; epoch_time: 2.93 sec; 
FastEstimator-Eval: step: 980; epoch: 10; base_ce: 1.8853257; adv_ce: 2.003

FastEstimator-Test: step: 980; epoch: 11; clean_accuracy: 0.3344; adversarial_accuracy: 0.276; 


In [3]:
model.state_dict()

OrderedDict([('module.conv1.weight',
              tensor([[[[ 1.4963e-01, -1.8061e-01,  1.3808e-01],
                        [ 1.1095e-01,  2.3134e-02,  1.6698e-01],
                        [-1.5297e-01,  1.7768e-01, -7.0454e-02]],
              
                       [[-5.8569e-02, -1.6035e-01,  1.9397e-01],
                        [-9.1226e-02, -4.2557e-02,  1.6467e-01],
                        [ 4.5859e-02, -1.7637e-01, -1.4366e-02]],
              
                       [[-5.0999e-02, -4.5869e-02,  1.6855e-01],
                        [ 7.5181e-02,  9.2338e-02,  1.6033e-01],
                        [ 1.0665e-01, -3.0521e-02,  1.0903e-01]]],
              
              
                      [[[-1.2066e-01, -2.4289e-02,  1.1210e-01],
                        [-2.1171e-02, -5.7936e-02, -1.4246e-01],
                        [-4.8552e-02, -3.3261e-02, -5.2477e-03]],
              
                       [[-6.1478e-02, -1.1886e-01,  1.4134e-01],
                        [-6.2880e-02, 